## 1. Import MNIST data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## 2. Create comptuation graph

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
print(tf.__version__) 

1.1.0


### 2.1 Define the neural network structure

In [4]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

### 2.2. Create placeholders for training data

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="InputImages")
y = tf.placeholder(tf.int64, shape=(None), name="TrueLabel")

### 2.3 Create neural network with two hidden layers and one output layer

In [6]:
hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
output  = tf.layers.dense(hidden2, n_outputs, name='output')

### 2.4. Define cost function

In [7]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=output)
cost = tf.reduce_mean(cross_entropy)

### 2.5. Define optimizer

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training  = optimizer.minimize(cost)

### 2.5. Define how to calculate  prediction accuracy

In [9]:
compare = tf.nn.in_top_k(output,y,1)
accuracy = tf.reduce_mean(tf.cast(compare,tf.float32))

## 3. Execute the computation graph

In [10]:
init = tf.global_variables_initializer()

In [11]:
saver = tf.train.Saver()  # save our trained model parameters to disk

Define mini-batch size.

In [12]:
n_epochs = 40
batch_size = 50
n_batches = int(np.ceil(mnist.train.num_examples/batch_size))

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in np.arange(n_epochs):
        for batch_index in np.arange(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training,feed_dict={X:X_batch, y:y_batch})
        accuracy_train = sess.run(accuracy,feed_dict={X:X_batch, y:y_batch})
        accuracy_test  = sess.run(accuracy,feed_dict={X:mnist.test.images, y:mnist.test.labels})
        
        print(epoch, "Train accuracy:", accuracy_train, "Test accuracy:", accuracy_test)
    
    save_patch = saver.save(sess,'./my_final_model.ckpt')

0 Train accuracy: 0.88 Test accuracy: 0.9022
1 Train accuracy: 0.94 Test accuracy: 0.9236
2 Train accuracy: 0.94 Test accuracy: 0.9323
3 Train accuracy: 0.98 Test accuracy: 0.9347
4 Train accuracy: 0.96 Test accuracy: 0.9418
5 Train accuracy: 0.94 Test accuracy: 0.9443
6 Train accuracy: 0.98 Test accuracy: 0.9486
7 Train accuracy: 0.98 Test accuracy: 0.951
8 Train accuracy: 0.98 Test accuracy: 0.9548
9 Train accuracy: 1.0 Test accuracy: 0.9572
10 Train accuracy: 1.0 Test accuracy: 0.9582
11 Train accuracy: 0.98 Test accuracy: 0.9608
12 Train accuracy: 0.98 Test accuracy: 0.9632
13 Train accuracy: 1.0 Test accuracy: 0.9639
14 Train accuracy: 0.98 Test accuracy: 0.9647
15 Train accuracy: 1.0 Test accuracy: 0.9659
16 Train accuracy: 1.0 Test accuracy: 0.9674
17 Train accuracy: 1.0 Test accuracy: 0.9676
18 Train accuracy: 0.98 Test accuracy: 0.9681
19 Train accuracy: 1.0 Test accuracy: 0.9688
20 Train accuracy: 1.0 Test accuracy: 0.9695
21 Train accuracy: 1.0 Test accuracy: 0.9696
22 Train

## 4. Make predictions

In [15]:
with tf.Session() as sess:
    saver.restore(sess, "./my_final_model.ckpt") # or better, use save_path
    X_new_scaled = mnist.test.images[:20]
    Z = output.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_final_model.ckpt


In [16]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", mnist.test.labels[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


## 5. Visualize the computation graph

In [17]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [22]:
show_graph(tf.get_default_graph())